---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [24]:
import pandas as pd
import numpy as np
from functools import reduce


energy = pd.read_excel('Energy Indicators.xls', skiprows=16)
energy.drop(energy.columns[[0,1]], axis=1, inplace=True)
energy = energy[:228]
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
for i, row in energy.iterrows():
    if energy.at[i,'Energy Supply'] != '...':
        energy.at[i,'Energy Supply']*=1000000
energy.iloc[0]['Energy Supply'] = 'Gigajoules'
energy.loc[energy['Energy Supply per Capita'] == '...','Energy Supply per Capita'] = np.nan 
energy.loc[energy['Energy Supply'] == '...','Energy Supply'] = np.nan 
energy['Country'] = energy['Country'].str.replace(r"\(.*\)","").str.replace('\d+', '').replace({"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"})

GDP = pd.read_csv('world_bank.csv', skiprows=4)
GDP = GDP.rename(columns = {'Country Name':'Country', 'Country Code':'Ctry Code'})
GDP['Country'] = GDP['Country'].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"})
    
ScimEn = pd.read_excel('scimagojr-3.xlsx')
ScimEn = ScimEn[:15]
    
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Country'],
                                            how='outer'), [energy, GDP, ScimEn])
df_merged.drop(df_merged.loc[:, '1960':'2005'], inplace=True, axis=1)
df_merged = df_merged[np.isfinite(df_merged['Rank'])]
df_merged = df_merged.set_index('Country')
df_merged = df_merged[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
df_merged['Rank'] = df_merged['Rank'].apply(np.int64)
df_merged['Documents'] = df_merged['Documents'].apply(np.int64)
df_merged['Citable documents'] = df_merged['Citable documents'].apply(np.int64)
df_merged['Citations'] = df_merged['Citations'].apply(np.int64)
df_merged['Self-citations'] = df_merged['Self-citations'].apply(np.int64)
df_merged['H index'] = df_merged['H index'].apply(np.int64)
df_merged['Energy Supply'] = df_merged['Energy Supply'].apply(np.float64)
df_merged['Energy Supply per Capita'] = df_merged['Energy Supply per Capita'].apply(np.float64)
df_merged['% Renewable'] = df_merged['% Renewable'].apply(np.float64)

def answer_one():
    return df_merged

answer_one()

Rank  Documents  Citable documents  Citations  \
Country                                                             
Australia             14       8831               8725      90765   
Brazil                15       8668               8596      60702   
Canada                 6      17899              17620     215003   
China                  1     127050             126767     597237   
France                 9      13153              12973     130632   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
Italy                 11      10964              10794     111850   
Japan                  3      30504              30287     223024   
South Korea           10      11983              11923     114675   
Russian Federation     5      18534              18301      34266   
Spain                 12       9428               9330     123336   
United Kingdom         4      20944              20357     206091   
United States          2      96661              94747     792274   
Iran                  13       8896               8819      57470   

                    Self-citations  Citations per document  H index  \
Country                                                               
Australia                    15606                   10.28      107   
Brazil                       14396                    7.00       86   
Canada                       40930                   12.01      149   
China                       411683                    4.70      138   
France                       28601                    9.93      114   
Germany                      27426                    8.26      126   
India                        37209                    8.58      115   
Italy                        26661                   10.20      106   
Japan                        61554                    7.31      134   
South Korea                  22595                    9.57      104   
Russian Federation           12422                    1.85       57   
Spain                        23964                   13.08      115   
United Kingdom               37874                    9.84      139   
United States               265436                    8.20      230   
Iran                         19125                    6.46       72   

                    Energy Supply  Energy Supply per Capita  % Renewable  \
Country                                                                    
Australia            5.386000e+09                     231.0    11.810810   
Brazil               1.214900e+10                      59.0    69.648030   
Canada               1.043100e+10                     296.0    61.945430   
China                1.271910e+11                      93.0    19.754910   
France               1.059700e+10                     166.0    17.020280   
Germany              1.326100e+10                     165.0    17.901530   
India                3.319500e+10                      26.0    14.969080   
Italy                6.530000e+09                     109.0    33.667230   
Japan                1.898400e+10                     149.0    10.232820   
South Korea          1.100700e+10                     221.0     2.279353   
Russian Federation   3.070900e+10                     214.0    17.288680   
Spain                4.923000e+09                     106.0    37.968590   
United Kingdom       7.920000e+09                     124.0    10.600470   
United States        9.083800e+10                     286.0    11.570980   
Iran                          NaN                       NaN          NaN   

                            2006          2007          2008          2009  \
Country                                                                      
Australia           1.021939e+12  1.060340e+12  1.099644e+12  1.119654e+12   
Brazil              1.845080e+12  1.957118e+12  2.056809e+12  2.054215e+12   
Canada              1.564469e+12 

###### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [3]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [106]:
def answer_two():
    df_lost = pd.merge(pd.merge(energy, GDP, on='Country', how='outer'), ScimEn, on='Country', how='outer')
    df_merged = pd.merge(pd.merge(energy, GDP, on='Country'), ScimEn, on='Country')
    return len(df_lost)-len(df_merged)

answer_two()

294

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [5]:
def answer_three():
    Top15 = answer_one()
    avgGDP = pd.Series((Top15[['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']].mean(axis=1)), index=[Top15.index])
    return avgGDP

answer_three()

Country
Australia             1.164043e+12
Brazil                2.189794e+12
Canada                1.660647e+12
China                 6.348609e+12
France                2.681725e+12
Germany               3.493025e+12
India                 1.769297e+12
Italy                 2.120175e+12
Japan                 5.542208e+12
South Korea           1.106715e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
United Kingdom        2.487907e+12
United States         1.536434e+13
Iran                  4.441558e+11
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [6]:
def answer_four():
    Top15 = answer_one()
    avgGDP = answer_three()
    count = 0
    sixth = 0
    for i in avgGDP.index:
        count = count + 1
        maxGDP = avgGDP.argmax()
        if count == 6:
            sixth = maxGDP
        avgGDP = avgGDP.drop(maxGDP)
    chng = (Top15['2015']['United Kingdom'] - Top15['2006']['United Kingdom'])
    return chng

answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [7]:
def answer_five():
    Top15 = answer_one()
    tot = Top15.loc[:,'Energy Supply per Capita'].sum() + 119
    meanESC = tot/15
    return meanESC

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [8]:
def answer_six():
    Top15 = answer_one()
    max_percent = Top15.loc[:,'% Renewable'].max()
    max_country = Top15.loc[:,'% Renewable'].argmax()
    tup = (max_country, max_percent)
    return tup

answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [25]:
def answer_seven():
    Top15 = answer_one()
    Top15R = Top15.copy()
    Top15R['Ratio'] = Top15R['Self-citations']/Top15['Citations']
    max_ratio = Top15R.loc[:,'Ratio'].max()
    max_country = Top15R.loc[:,'Ratio'].argmax()
    tup = (max_country, max_ratio)
    return tup

answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [41]:
def answer_eight():
    Top15 = answer_one()
    Top15Pop = Top15.copy()
    Top15Pop['Pop Estimate'] = Top15Pop['Energy Supply']/Top15Pop['Energy Supply per Capita']
    Top15Pop['Pop Estimate']['Iran'] = 77075630
    count = 0
    third = 0
    for i in Top15Pop['Pop Estimate']:
        count = count + 1
        maxpop = Top15Pop['Pop Estimate'].argmax();
        Top15Pop = Top15Pop.drop(maxpop)
        if count == 3:
            third = maxpop
    return third

answer_eight()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [121]:
def answer_nine():
    Top15 = answer_one()
    Top15Pop = Top15.copy()
    Top15Pop['Pop Estimate'] = Top15Pop['Energy Supply']/Top15Pop['Energy Supply per Capita']
    Top15Pop['Pop Estimate']['Iran'] = 9172000000/119
    Top15Pop['Citable docs per Capita'] = Top15Pop['Citable documents']/Top15Pop['Pop Estimate']
    corr = Top15Pop[['Energy Supply per Capita','Citable docs per Capita']].corr().ix['Energy Supply per Capita','Citable docs per Capita']
    return corr

answer_nine()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.79019354883008719

In [48]:
#def plot9():
#    import matplotlib as plt
#    %matplotlib inline
    
#    Top15 = answer_one()
#    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
#    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
#    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [55]:
def answer_ten():
    Top15 = answer_one()
    Top15_10 = Top15.copy()
    Top15_10['% Renewable']['Iran'] = 5.71
    medperRen = Top15_10.loc[:,'% Renewable'].median()
    Top15_10.sort_values('Rank', inplace=True)
    above_or_below = []
    for i in Top15_10['% Renewable']:
        if i < medperRen:
            above_or_below.append(0)
        else:
            above_or_below.append(1)
    HighRenew = pd.Series((above_or_below), index=[Top15_10.index])
    return HighRenew

answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [74]:
def answer_eleven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15Pop = Top15.copy()
    Top15Pop['Pop Estimate'] = Top15Pop['Energy Supply']/Top15Pop['Energy Supply per Capita']
    Top15Pop['Pop Estimate']['Iran'] = 9172000000/119
    Top15Pop['Continent'] = Top15Pop.index.map(lambda x: ContinentDict[x])
    continents = Top15Pop.groupby('Continent')['Pop Estimate'].agg(['size', 'sum','mean','std'])
    continents['size'] = continents['size'].apply(np.float64)
    return continents

answer_eleven()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


size           sum          mean           std
Continent                                                    
Asia            5.0  2.898666e+09  5.797333e+08  6.790979e+08
Australia       1.0  2.331602e+07  2.331602e+07           NaN
Europe          6.0  4.579297e+08  7.632161e+07  3.464767e+07
North America   2.0  3.528552e+08  1.764276e+08  1.996696e+08
South America   1.0  2.059153e+08  2.059153e+08           NaN

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [120]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15Cut = Top15.copy()
    Top15Cut['% Renewable']['Iran'] = 5.71
    Top15Cut['Continent'] = Top15Cut.index.map(lambda x: ContinentDict[x])
    Top15Cut = Top15Cut.reset_index()
    Top15Cut['bins'] = pd.cut(Top15Cut['% Renewable'], 5)
    continents = Top15Cut.groupby(['Continent','bins']).size()
    return continents

answer_twelve()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Continent      bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [104]:
def answer_thirteen():
    Top15 = answer_one()
    Top15Pop = Top15.copy()
    Top15Pop['Pop Estimate'] = Top15Pop['Energy Supply']/Top15Pop['Energy Supply per Capita']
    Top15Pop['Pop Estimate']['Iran'] = 9172000000/119
    Top15Pop['commas'] = Top15Pop['Pop Estimate'].map('{:,}'.format)
    PopEst = pd.Series((Top15Pop['commas']), index=Top15Pop.index)
    return PopEst

answer_thirteen()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Country
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
France                  63,837,349.39759036
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
South Korea            49,805,429.864253394
Russian Federation            143,500,000.0
Spain                    46,443,396.2264151
United Kingdom         63,870,967.741935484
United States          317,615,384.61538464
Iran                    77,075,630.25210084
Name: commas, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!